# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e compramos e vendemos commodities:
- Soja, Milho, Trigo, Petróleo, etc.

Precisamos pegar na internet, de forma automática, a cotação de todas as commodites e ver se ela está abaixo do nosso preço ideal de compra. Se tiver, precisamos marcar como uma ação de compra para a equipe de operações.

Base de Dados: https://drive.google.com/drive/folders/1O9aE_Hen5smf_a6TsbVF6DM0ms4Ykj6s?usp=share_link

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [10]:
!pip install selenium
!pip install pandas
!pip install openpyxl
!pip install numpy

In [ ]:
# adicionar dentro dos arquivos do python
    # firefox -> geckodriver
    # chrome -> chromedriver

# Passo a passo

# Passo 1: Entrar na internet (abrir o navegador)
from selenium import webdriver
navegador = webdriver.Chrome()
navegador.get("https://www.google.com/")



In [ ]:
# Passo 2: importar a base de dados
import pandas as pd

tabela = pd.read_excel("commodities.xlsx")
display(tabela)

In [ ]:
# import unicodedata
# produto = unicodedata.normalize("NFKD", produto).encode("ascii", "ignore")
# mais facil de fazer as substituições das palavras especiais, em vez de colocar um monte de replace

for linha in tabela.index:
    #descobrir qual e o produto, loc = localização, depois da linha colocar a coluna que ele tem que editar entre ""  que no caso e o produto 
    produto = tabela.loc[linha, "Produto"]
      
    # entrar no site melhor "https://www.melhorcambio.com/{produto}-hoje", entre {} o produto pra sempre substituir para o produto que voce deseja
    link = f"https://www.melhorcambio.com/{produto}-hoje"
    link = link.replace("ó", "o").replace("ã", "a").replace("á", "a").replace(
    "ç", "c").replace("ú", "u").replace("é", "e")
    print(link)
    navegador.get(link)

    # pegar a cotação do produto, value e o atributo do numero da cotação, replace para tirar o . e substiuir a , por ., para trasnformar em um numero que o python consiga fazer conta 1.257,43 = 1257.43, float para mudar a cotacao para um numero 
    cotacao = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')
    cotacao = cotacao.replace(".", "").replace(",", ".")
    cotacao = float(cotacao)
    print(cotacao)
    # loc = localização, depois da linha colocar a coluna que ele tem que editar entre "" 
    tabela.loc[linha, "Preço Atual"] = cotacao
    
    


In [ ]:
# Passo 6: Decidir quais produtos a gente vai comprar
# preencher a coluna de comprar
tabela["Comprar"] = tabela["Preço Atual"] < tabela["Preço Ideal"]
display(tabela)


In [ ]:
# Fechar o navegador
navegador.quit()

In [ ]:
# Passo 7: Exportar a base de dados atualizado
# exportar a base para o excel
tabela.to_excel("commodities_atualizado.xlsx", index=False)
# index=False, para exportar sem o indice 